In [146]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

import sqlite3

<IPython.core.display.Javascript object>

In [167]:
from simpledbf import Dbf5
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import Image
import pandas as pd
import numpy as np
pd.options.display.max_colwidth = 200


class Database():
    
    def __init__(self, db_name): self.connection = self.connect(db_name)
    
    def read(self, sql): pass
    
    def connect(self,db_name): pass
    
    def insert(self, table_name, dataframe): print("yaa, inserted, get some!")
        
    def disconnect(self): pass
        
    def __del__(self): self.disconnect()
        

class SQLiteDB(Database):
    
    def connect(self, db_name): return sqlite3.connect(db_name)
    
    def read(self, sql): return pd.read_sql_query(sql, self.connection,
                                                 )#index_col="id")
    
    def insert(self, table_name, dataframe):
        dataframe.to_sql(table_name, self.connection,
                         if_exists="append", index = False)
        
    def disconnect(self): self.connection.close()
        
db_path = "./data/db.sqlite"
db = SQLiteDB(db_path)

In [169]:
def plot(df):
    lines = [627,652,675,698,725,754]
    x = df["CenterY"]/10000
    y = df["CenterX"]/10000
    s = df["GISHektar"]/10
    print("Total fires: {}".format(len(df.index)))
    #display(df.head())
    plt.figure(figsize=(9,9))
    plt.scatter(x,y, s, color = 'orange', linewidths = 2)
    plt.hlines(lines,30,90, color = 'green')
    plt.axis('equal')
    plt.show()
    
def show_image(area): display(Image(filename='images/area_{}.png'.format(area)))
    
def write(db, df): db.insert('skogsbrand', df)
def read(db): return db.read("SELECT * FROM skogsbrand")

df = read(db)
df.describe()
df = df[df.GISHektar > 1]
#df = df[df.quality != np.nan]
#df = df[df.zone == np.nan]
print("Total fires: {}".format(len(df.index)))
display(df)
#write(db,df)

plot(df)


TypeError: '>' not supported between instances of 'str' and 'int'

In [168]:
def read_dbf(): return Dbf5('data/brand.dbf').to_dataframe()
def setup_columns(df):
    df['link'] = "https://kartor.skogsstyrelsen.se/kartorapp/?startapp=skogligagrunddata&x="+df.CenterX.astype(str)+"&y="+df.CenterY.astype(str)+"&scale=37502.2872&bg=KARTA"
    df['quality'] = np.nan
    df['comments'] = np.nan
    df['zone'] = np.nan
    df['road_distance'] = np.nan
    df['burntX'] = np.nan
    df['burntY'] = np.nan
    df['unburntX'] = np.nan
    df['unburntY'] = np.nan
    return df
def be_careful():
    df = read_dbf()
    df = setup_columns(df)
    db = SQLiteDB("./data/db.sqlite")
    db.insert('skogsbrand', df)

be_careful()

